In [ ]:
from mytest import utils as iu
from rfdiffusion.potentials.manager import PotentialManager
import logging
import torch.nn.functional as nn
from rfdiffusion import util

from rfdiffusion.model_input_logger import pickle_function_call

import os, time, pickle
import torch

import logging
from rfdiffusion.util import writepdb_multi, writepdb
from hydra.core.hydra_config import HydraConfig
import numpy as np
import random

from mytest.mymodel import *

from hydra import initialize, compose
from omegaconf import OmegaConf

def make_deterministic(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


with initialize(version_base=None, config_path="../config/inference"):
    conf = compose(config_name="base")

#conf.inference.ckpt_override_path= "../Train/weights_base3/best_model.pth"

# 调用主函数
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

log = logging.getLogger(__name__)
if conf.inference.deterministic:
    make_deterministic()

# Check for available GPU and print result of check
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
    log.info(f"Found GPU with device_name {device_name}. Will run RFdiffusion on {device_name}")
else:
    log.info("////////////////////////////////////////////////")
    log.info("///// NO GPU DETECTED! Falling back to CPU /////")
    log.info("////////////////////////////////////////////////")

# Initialize sampler and target/contig.
sampler = MySampler(conf,device=device)  # 使用传入的 device 参数
sampler.model.eval()
target = iu.Target(conf.design_config,device=device)
partial_T = conf.design_config.partial_T

x_init, seq_init, _, _ = sampler.sample_init_forward(B=B,L=L)


# # Loop over number of designs to sample.
# design_startnum = sampler.inf_conf.design_startnum
# if sampler.inf_conf.design_startnum == -1:
#     existing = glob.glob(sampler.inf_conf.output_prefix + "*.pdb")
#     indices = [-1]
#     for e in existing:
#         print(e)
#         m = re.match(".*_(\d+)\.pdb$", e)
#         print(m)
#         if not m:
#             continue
#         m = m.groups()[0]
#         indices.append(int(m))
#     design_startnum = max(indices) + 1


    

/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/rfdiffusion/util.py:253: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /pytorch/aten/src/ATen/native/Cross.cpp:62.)
  Z = torch.cross(Xn, Yn)
/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/dgl/backend/pytorch/sparse.py:104: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/home/fit/lulei/WORK/.conda/envs/SE3nv/lib/python3.9/site-packages/dgl/backend/pytorch/sparse.py:128: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dZ):
/home/fit/lulei/WORK/.conda/en

Reading models from /WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/tests/../../models
This is inf_conf.ckpt_path
/WORK/PUBLIC/lulei_work/xjt/Protein_design/RFdiffusion/mytest/tests/../../models/Base_ckpt.pt


TypeError: __init__() got an unexpected keyword argument 'device'

In [3]:
out_prefix = "samples/samples4/design"

for i_des in range(design_startnum, design_startnum + sampler.inf_conf.num_designs):
    if conf.inference.deterministic:
        make_deterministic(i_des)

    start_time = time.time()
    # out_prefix = f"{sampler.inf_conf.output_prefix}_{i_des}"
    log.info(f"Making design {out_prefix}")
    if sampler.inf_conf.cautious and os.path.exists(out_prefix + ".pdb"):
        log.info(
            f"(cautious mode) Skipping this design because {out_prefix}.pdb already exists."
        )
        continue
    part_t = 200
    B,L = 2, 60
    x_init, seq_init, _, _ = sampler.sample_init_forward(B=B,L=L)
    denoised_xyz_stack = []
    px0_xyz_stack = []
    seq_stack = []
    plddt_stack = []
  
    x_t = torch.clone(x_init)
    seq_t = torch.clone(seq_init)
    bond_mat_t = torch.zeros((B, L, L), device=device, dtype=torch.float32)
    # Loop over number of reverse diffusion time steps.
    
    for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
    # for t in range(part_t, sampler.inf_conf.final_step - 1, -1):
        print("t"*100, t)
        print("shurud seq_t",seq_t.isnan().any())
        px0, x_t, seq_t, bond_mat,plddt = sampler.sample_step(
            t=t, x_t=x_t, seq_t=seq_t, bond_mat_t=bond_mat_t ,final_step=sampler.inf_conf.final_step
        )
        seq_real = torch.argmax(seq_t[:,:,:20], dim=-1)
        px0_xyz_stack.append(px0)
        denoised_xyz_stack.append(x_t)
        seq_stack.append(seq_real)
        plddt_stack.append(plddt[0])  # remove singleton leading dimension
        # print("seqtshape",seq_t.shape)
        print("seq_t",seq_real)
        # print("px0",px0[:,:,1,:])
        # print("plddt",plddt.item)
        # print("seq",seq_real)
    # Flip order for better visualization in pymol
    denoised_xyz_stack = torch.stack(denoised_xyz_stack)
    denoised_xyz_stack = torch.flip(
        denoised_xyz_stack,
        [
            0,
        ],
    )
    px0_xyz_stack = torch.stack(px0_xyz_stack)
    px0_xyz_stack = torch.flip(
        px0_xyz_stack,
        [
            0,
        ],
    )
    # For logging -- don't flip
    plddt_stack = torch.stack(plddt_stack)
    seq_stack = torch.stack(seq_stack)
    break

With this beta schedule (linear schedule, beta_0 = 0.01, beta_T = 0.07), alpha_bar_T = 0.00027539944858290255
tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt 200
shurud seq_t tensor(False, device='cuda:0')
tensor([[[    -2.3313,      1.0096,     -1.3348,  ...,     -2.3289,
               0.7102,      2.4650],
         [    -1.8027,      0.3547,      1.2327,  ...,      0.5876,
               2.8786,      2.9232],
         [    -2.5383,     -2.2359,     -2.5781,  ...,      0.4720,
               2.0428,      2.7363],
         ...,
         [    -2.0641,      0.4624,      1.5338,  ...,      1.1774,
               4.1035,      2.6825],
         [    -0.9092,     -1.0325,     -2.0253,  ...,      0.1423,
               1.1303,      0.8116],
         [    -2.0141,     -0.3492,     -0.0687,  ...,     -1.6806,
               2.0579,     -1.0594]],

        [[    -0.2480,      0.0161,     -2.2384,  ...,     -3.1069,
              -1.3842,     

In [8]:
# final_seqs = seq_stack[-1]
# final_xyz_stack = denoised_xyz_stack[0]
print(seq_stack[-1])
batch_size = denoised_xyz_stack.size(1)


os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
for i in range(batch_size):
    final_seq = seq_stack[-1,i]
    final_xyz = denoised_xyz_stack[0,i]
    
    # Output glycines, except for motif region
    # final_seq = torch.where(
    #     torch.argmax(seq_init[i], dim=-1) == 21, 7, torch.argmax(seq_init[i], dim=-1)
    # )  # 7 is glycine

    bfacts = torch.ones_like(final_seq.squeeze())
    # make bfact=0 for diffused coordinates
    bfacts[torch.where(torch.argmax(seq_init[i], dim=-1) == 21, True, False)] = 0
    
    # pX0 last step
    batch_out_prefix = f"{out_prefix}_{i}"
    out = f"{batch_out_prefix}.pdb"

    # Now don't output sidechains
    writepdb(
        out,
        final_xyz[:, :17],
        final_seq,
        sampler.binderlen,
        chain_idx=sampler.chain_idx,
        bfacts=bfacts,
    )

    # run metadata
    trb = dict(
        config=OmegaConf.to_container(sampler._conf, resolve=True),
        plddt=plddt_stack[:, i].cpu().numpy(),
        device=torch.cuda.get_device_name(torch.cuda.current_device())
        if torch.cuda.is_available()
        else "CPU",
        time=time.time() - start_time,
    )
    if hasattr(sampler, "contig_map"):
        for key, value in sampler.contig_map.get_mappings().items():
            trb[key] = value
    with open(f"{batch_out_prefix}.trb", "wb") as f_out:
        pickle.dump(trb, f_out)

    if sampler.inf_conf.write_trajectory:
        # trajectory pdbs
        traj_prefix = (
            os.path.dirname(batch_out_prefix) + "/traj/" + os.path.basename(batch_out_prefix)
        )
        os.makedirs(os.path.dirname(traj_prefix), exist_ok=True)

        out = f"{traj_prefix}_Xt-1_traj.pdb"
        writepdb_multi(
            out,
            denoised_xyz_stack[:, i],
            bfacts,
            final_seq.squeeze(),
            use_hydrogens=False,
            backbone_only=False,
            chain_ids=sampler.chain_idx,
        )

        out = f"{traj_prefix}_pX0_traj.pdb"
        writepdb_multi(
            out,
            px0_xyz_stack[:, i],
            bfacts,
            final_seq.squeeze(),
            use_hydrogens=False,
            backbone_only=False,
            chain_ids=sampler.chain_idx,
        )

log.info(f"Finished design in {(time.time()-start_time)/60:.2f} minutes")


tensor([[19, 15, 18, 16,  9,  2, 15,  6, 18, 14, 19,  2,  6, 18,  6,  7, 11, 14,
         14,  5, 19, 11, 18, 18, 18,  7,  6, 14, 11, 17, 18, 18, 16,  7, 14, 14,
         14,  5, 15,  6, 14,  7,  9, 14,  6,  7,  7, 19,  7, 19, 18, 18,  8, 18,
          6,  7,  6, 19, 18,  6],
        [ 7,  7, 19, 19, 18, 19,  7, 18,  6,  6, 14, 18,  7,  7,  6,  7,  7, 17,
          6,  7,  7,  7, 19, 14, 19, 14,  7,  6,  2, 15, 14,  7, 18, 14, 18,  6,
          1, 19,  7,  7,  6, 14, 14, 18, 14, 19, 18, 18,  7, 17,  6,  6, 14,  6,
         18,  6,  7, 14, 18,  6]], device='cuda:0')


In [4]:
# Save outputs
os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
final_seq = seq_stack[-1]

# Output glycines, except for motif region
# final_seq = torch.where(
#     torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
# )  # 7 is glycine

bfacts = torch.ones_like(final_seq.squeeze())
# make bfact=0 for diffused coordinates
bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0
# pX0 last step
out = f"{out_prefix}.pdb"

# Now don't output sidechains
writepdb(
    out,
    denoised_xyz_stack[0, :, :4],
    final_seq,
    sampler.binderlen,
    chain_idx=sampler.chain_idx,
    bfacts=bfacts,
)

# run metadata
trb = dict(
    config=OmegaConf.to_container(sampler._conf, resolve=True),
    plddt=plddt_stack.cpu().numpy(),
    device=torch.cuda.get_device_name(torch.cuda.current_device())
    if torch.cuda.is_available()
    else "CPU",
    time=time.time() - start_time,
)
if hasattr(sampler, "contig_map"):
    for key, value in sampler.contig_map.get_mappings().items():
        trb[key] = value
with open(f"{out_prefix}.trb", "wb") as f_out:
    pickle.dump(trb, f_out)

if sampler.inf_conf.write_trajectory:
    # trajectory pdbs
    traj_prefix = (
        os.path.dirname(out_prefix) + "/traj/" + os.path.basename(out_prefix)
    )
    os.makedirs(os.path.dirname(traj_prefix), exist_ok=True)

    out = f"{traj_prefix}_Xt-1_traj.pdb"
    writepdb_multi(
        out,
        denoised_xyz_stack,
        bfacts,
        final_seq.squeeze(),
        use_hydrogens=False,
        backbone_only=False,
        chain_ids=sampler.chain_idx,
    )

    out = f"{traj_prefix}_pX0_traj.pdb"
    writepdb_multi(
        out,
        px0_xyz_stack,
        bfacts,
        final_seq.squeeze(),
        use_hydrogens=False,
        backbone_only=False,
        chain_ids=sampler.chain_idx,
    )

log.info(f"Finished design in {(time.time()-start_time)/60:.2f} minutes")


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
print(denoised_xyz_stack[12,0,:,1,:])
print(denoised_xyz_stack[4,0,:,1,:])

tensor([[ 0.6988, -5.7258, -0.3178],
        [ 1.5968, -1.2029,  3.5006],
        [-1.0109,  1.8305,  1.4969],
        [-5.2286,  4.3836, 10.8067],
        [ 3.7435, -1.4711,  5.0631],
        [-1.5842,  1.0947,  4.5744],
        [-3.8826, -0.0143,  1.9285],
        [ 1.2966,  4.8627,  1.6958],
        [-3.3198, -9.0396,  1.6099],
        [ 6.1156, -3.6806, -5.9215],
        [-0.8840, -1.5614, -1.7241],
        [-3.2642,  1.7146, -4.6148],
        [ 0.2229,  2.5003, -1.7190],
        [-5.3268,  0.5873, -2.6419],
        [ 6.3170, -4.5131, -5.4884],
        [ 0.3481,  4.9903, -5.9112],
        [ 1.1136, -1.8126,  4.2289],
        [ 1.6004, -6.1372, -0.1157]], device='cuda:0')
tensor([[ 0.6988, -5.7258, -0.3178],
        [ 1.5968, -1.2029,  3.5006],
        [-1.0109,  1.8305,  1.4969],
        [-5.2286,  4.3836, 10.8067],
        [ 3.7435, -1.4711,  5.0631],
        [-1.5842,  1.0947,  4.5744],
        [-3.8826, -0.0143,  1.9285],
        [ 1.2966,  4.8627,  1.6958],
        [-3.3198, -9